<a href="https://colab.research.google.com/github/WetSuiteLeiden/example-notebooks/blob/main/specific-experiments/find-references/find_references.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# (only) in colab, run this first to install wetsuite from (the most recent) source.   For your own setup, see wetsuite's install guidelines.
!pip3 install -U wetsuite

# Purpose of this notebook



Seeing how well you can extract the more explicit references to laws and jurisprudence from relatively free-form text.

Because there are few standards, and the ones there are are rarely used consistently.

You probably want to skim [notes__legal_identifiers_and_references (notebook)](../../notes/notes__legal_identifiers_and_references.ipynb) first.

Optionally, take a glance at [find_wetnamen](../wetnamen/find_wetnamen.ipynb) extracting the names of laws,
including less-official references in real-world use.

Note that there are many more implicit references, such as documents referring to `de staatssecretaris`, `onze wijziging`, `die brief`, `de najaarsnota`, many of which  _could_ be reconciled well enough within their context, but require a more involved a mix of more NLP-like approaches, co-reference and beyond, that this example is not even attempting yet.

The amount of attention that deserves is usually only found in theses and such -- see e.g. [this one](https://scripties.uba.uva.nl/search?id=record_54697).

## What is our domain?

There will be references between documents like 
jurisprudence,
national law,
EU law, 
policy, 
and other official publications.

Extracting such references may be useful to varied tasks, such as 
- finding related documents by the references they share use, 
- estimate the generla topics a document covers,
- estimate what a document intends to do, by the legal authority it draws on.

<!-- -->

Particularly ECLI marks what it is in the identifier itself, so is unambiguous. 

The full form of a jci is similarly recognizable.


Many other things fall on a gliding scale.
LJN may just look like some letters and numbers.
The same goes for CELEX except there are some letters in a predictable yet otherwise-unusual place.
Such details metter if you want to avoid false positives.

Abbreviated references are more easily missed (false negatives), or lead to false positives.

Natural-language references may see a lot of _variation_ in how they are written down,
but their relative verbosity means that most cases are still fairly unambiguous.





### Some practical notes


TODO: MOVE WHAT IS RELEVANT TO NOTES_IDENTIFIERS


**Nerdnotes:**
* You could look for very specific text patterns, 
  * but things like regexps are likely to break under even minor variation and/or become a mess to understan (see much of the current implementation)
  * a good tradeoff might be something that lets you express the parts and how they combine - such as classical classical formal grammars - see acronyms like EBNF, CFGs, and in practice something like [PEG](https://en.wikipedia.org/wiki/Parsing_expression_grammar)s may be more flexible

* There are some things that are essentially global aliases, shared through the community
  * e.g. referring to part of `Awb` is _very_ likely to be _Algemene wet bestuursrecht_, [BWBR0005537](https://wetten.overheid.nl/BWBR0005537)
  * and some references into this will be common to the point of almost being concepts in themselves
  
* Then there are what are essentially local aliases. Consider:
  - `"Ingevolge artikel 2, eerste lid, van de Wet op de adeldom (hierna: de Wet)"`
  - `"the European Convention on Human Right and Fundamental Freedoms (hereinafter 'the Convention')"`
  * which are important if you want to resolve _"article 6 of the Convention"_


**Practicalities**
* Some of the data we have is marked up, meaning there are links that are to a formal identifier, and have a less-formal name. 
  * for example, find `extref` tags in BWB and CVDR's XML form

* Some of the references are nicely in metadata fields. Knowing that can act as disambiguation of the field's value in a way we cannot easily do in free-flowing text.

* We also just have text that happens to be references - e.g. in free-flowing text.


* perhaps the most important is actually that it is often not clear where the reference ends. A simple example:

    TODO


**Practicalities we cannot cover easily, or at all:**
* References would be referring to the version valid at the time of reference, though that should be resolvable

* You may find a lot of **less-formal names for laws** from free-form text

* The same may not work as well for **policy** or **jurisprudence**. 
    * Say, "Rensing/Polak II" is practically unambiguous, even though it's **not** `ECLI:NL:HR:2005:AT4537`'s name
    * yet there are many links that will be named "Besluit", or another name that is unique only within that document and not something we should learn generally

* Not all references have the same function. Consider `"Commission Directive 2014/110/EU of 17 December 2014 amending Directive 2004/33/EC as regards ..."` or `"The substantive provisions of Directive 2004/33/EC were not affected by its amendment by Directive 2014/110/EU ..."` - it would not be hard to extract both references, but it is is up to you to notice that in terms of interpretation, one is referred to, and the other is only mentioned for completeness, or to decide there is a difference between mentions, supporing references, etc.



## Some tests

### Test what we can extract: More on the identifier end

In [3]:
import pprint

import wetsuite.helpers.patterns
import wetsuite.helpers.meta
import wetsuite.helpers.split
import wetsuite.helpers.net

In [2]:
matches = wetsuite.helpers.patterns.find_references( """
BWBR0006501

CVDR101405_1  CVDR101405/1  CVDR101405

ECLI:NL:HR:2005:AT4537   is an exampe of LJN in ECLI - and note it will get an overlapping detection
ECLI:EU:C:1998:27        is not detailed further, we currently focus on dutch only
ECLI:EU:F:2010:80
ECLI:EU:T:2012:426

kst-26643-144-h1        is not something you usually find in text
h-tk-20082009-7140-7144 so this is also disabled by default
ah-tk-20082009-2945
stcrt-2009-9231

32000L0060
32012A0424(01)
32009L0164R(01)          is a corrigendum
52018PC0033              is internal
11957E086                is a treaty
72013L0032AUT_202103576  is a national transposition (the added p
                                                    ECLI:NL:HR:1841:1
OJ L 69, 13.3.2013, p. 1
OJ L 168, 30.6.2009, p. 41–47 
Council Directive 93/42/EEC of 14 June 1993
Council Regulation (EEC) No 2658/87 

Stb. 2011, 35;

LJN: BO6106
LJN: AO7817, NJ 2005, 270
""", ljn=True, bekendmaking_ids=True) # by default does not look for LJN, because that easily leads to some false positives

for match in matches:
    #print( match )
    print('%r'%match['text'])
    for k, v in match.items():
        print( '    %20r:  %-r'%(k, v) )

'BWBR0006501'
                  'type':  'bwb'
                 'start':  1
                   'end':  12
                  'text':  'BWBR0006501'
'CVDR101405_1'
                  'type':  'cvdr'
                 'start':  14
                   'end':  26
                  'text':  'CVDR101405_1'
               'details':  {'workid': '101405', 'expressionid': '101405_1'}
'CVDR101405/1'
                  'type':  'cvdr'
                 'start':  28
                   'end':  40
                  'text':  'CVDR101405/1'
               'details':  {'workid': '101405', 'expressionid': '101405_1'}
'CVDR101405'
                  'type':  'cvdr'
                 'start':  42
                   'end':  52
                  'text':  'CVDR101405'
               'details':  {'workid': '101405', 'expressionid': None}
'ECLI:NL:HR:2005:AT4537'
                  'type':  'ecli'
                 'start':  54
                   'end':  76
                  'text':  'ECLI:NL:HR:2005:AT4537'
           

### Test: matching less-structured cases

In [3]:
# These have more varied form, see how well we do at that
for test_text in '''Kamerstukken II 2015/16, 34442, nr. 3, p. 7.
Kamerstukken I 1995/96, 23700, nr. 188b, p. 3.
Kamerstukken I 2014/15, 33802, C, p. 3.
Kamerstukken 2014/15, 33802, C, p. 3.  as a test of missing number
Kamerstukken II 1999/2000, 2000/2001, 2001/2002, 26 855.
Kamerstukken I 2000/2001, 26 855 (250, 250a); 2001/2002, 26 855 (16, 16a, 16b, 16c).
Handelingen II 2000/2001, blz. 4225–4227; 4277.
Handelingen I 2001/2002, zie vergadering d.d. 4 december 2001.
Bijlage bij Kamerstukken II 2015/16, 34389, nr. 9.
Wetsvoorstel partnerschapsregistratie (Kamerstukken II 1993/94-1996/97, 23761).
Wetsvoorstel partnerschapsregistratie (Kamerstukken 1993/94-1996/97, 23761)
Aanhangsel Handelingen, vergaderjaar 2022–2023, nr. 2427
Aanhangsel Handelingen II, vergaderjaar 2023-2024, nr. 766 
'''.splitlines():
    
    matches = wetsuite.helpers.patterns.find_references( test_text )
    if len(matches)==0:
        print( "\nDID NOT MATCH IN %r"%test_text)
    else:
        for match in matches:
            print('\nMatched: %r'%match['text'])
            for k, v in match.items():
                print( '    %20r:  %-r'%(k, v) )


Matched: 'Kamerstukken II 2015/16, 34442, nr. 3, p. 7'
                  'type':  'kamerstukken'
                 'start':  0
                   'end':  43
                  'text':  'Kamerstukken II 2015/16, 34442, nr. 3, p. 7'

Matched: 'Kamerstukken I 1995/96, 23700, nr. 188'
                  'type':  'kamerstukken'
                 'start':  0
                   'end':  38
                  'text':  'Kamerstukken I 1995/96, 23700, nr. 188'

Matched: 'Kamerstukken I 2014/15, 33802, C, p. 3'
                  'type':  'kamerstukken'
                 'start':  0
                   'end':  38
                  'text':  'Kamerstukken I 2014/15, 33802, C, p. 3'

Matched: 'Kamerstukken 2014/15, 33802, C, p. 3'
                  'type':  'kamerstukken'
                 'start':  0
                   'end':  36
                  'text':  'Kamerstukken 2014/15, 33802, C, p. 3'

Matched: 'Kamerstukken II 1999/2000, 2000'
                  'type':  'kamerstukken'
                 'start':  0

...even less structured...

In [4]:
looser_testcases = (
    'artikel 8:75a, eerste lid, tweede volzin, van de Awb',

    'artikel 5:9, aanhef en onder b, Awb',
    'artikel 8, aanhef en onder c, Wet bescherming persoonsgegevens (Wbp).',
    'artikel 10, tweede lid, aanhef en onder e, van de Wob',
    "artikel 4, tweede lid, aanhef en onder d, van het reglement van orde voor de ministerraad",
    "Artikel 10, tweede lid, aanhef en onder e van de. Wet openbaarheid van bestuur",
    "artikel 6:80 lid 1 aanhef en onder b BW",
    'artikel 4, aanhef en onder d en g, van de standaardvoorwaarden',
    'artikel 3.3, zevende lid, aanhef en onder i, Woo',
    'artikel 15, aanhef en onder a of c (oud) RWN',

    '  artikel 81q lid 3c Gemeentewet  ',
    'artikel 4:25, 4:35 van de Awb en artikel 10 van de ASV',
    'Wabo, art. 2.12, eerste lid, aanhef en onder a, sub 1\xBA',
    'artikel 79, aanhef en onder 6\xBA',
    'artikel 3.3, zevende lid jo. vijfde lid, aanhef en onder i, Woo',
    'artikel 142, eerste lid, aanhef en onder b (en derde lid), van het Wetboek van Strafvordering',
    'artikel 2, eerste, tweede, vijfde, en zesenzestigste lid',
    'art. 166 lid 1 in verbinding met art. 353 lid 1 Rv',
    'artikel 13, derde lid, van de Wet tarieven in burgerlijke zaken',
    'artikel 7, eerste lid, onderdeel a',
    'artikel 7, eerste lid, aanhef en onderdeel b', 

    'artikel 4, tweede lid, aanhef en onder d, van het reglement van orde voor de ministerraad',
    'artikel 5:9, aanhef en onder b, Awb',
    'artikel 8, aanhef en onder c, Wet bescherming persoonsgegevens (Wbp)',
    'artikel 10, tweede lid, aanhef en onder e, van de Wob',
    'artikel 4, tweede lid, aanhef en onder d, van het reglement van orde voor de ministerraad',
    'Artikel 10, tweede lid, aanhef en onder e van de Wet openbaarheid van bestuur',
    'artikel 6:80 lid 1 aanhef en onder b BW',
    'artikel 142, eerste lid, aanhef en onder b (en derde lid), van het Wetboek van Strafvordering',
    'artikel 4, aanhef en onder d en g, van de standaardvoorwaarden',
    'artikel 3.3, zevende lid, aanhef en onder i, Woo',
    'artikel 3.3, zevende lid jo. vijfde lid, aanhef en onder i, Woo',
    'artikel 79, aanhef en onder 6\xBA',
    'artikel 15, aanhef en onder a of c (oud) RWN',
    'Wabo, art. 2.12, eerste lid, aanhef en onder a, sub 1\xBA',
    'artikel 2, eerste, tweede, en vijfde lid, gestelde eisen voldoen. (Artikel 3, eerste lid, van de Tabaks- en rookwarenwet)',
    'artikel 2, eerste, tweede, vijfde, even zesenzestigste lid',

    'art 18 lid 2 en 4 Besluit kwaliteit; art 21 lid 1 sub a Regeling',
    'artikel 2, onderdeel 8, subonderdeel b, van Verordening 2018/1805', 
    '(Art. 81 RO)',
    "(81 WWB)",
    '(als bedoeld in art. 22 Rv.)',
    'artikel 166 Rv',
    'artikel 3:303 BW',
    'artikel 22 Rv',
)

In [5]:
for test_text in looser_testcases:
    matches = wetsuite.helpers.patterns.find_references( test_text, debug=0 )
    if len(matches)==0:
        print( 'NO MATCH IN %r'%test_text )
    else:
        #continue
        # Note that right now, we do not yet try to match the mentioned law -- that's on a TODO list
        print( '\n%r -->'%test_text )
        for match in matches:
            #pprint.pprint( match )
            for k, v in match['details'].items():
                print( '    %15s:  %-r'%(k, v) )


'artikel 2, onderdeel 8, subonderdeel b, van Verordening 2018/1805' -->
            artikel:  '2'
        aanhefonder:  'onderdeel b'

'artikel 8:75a, eerste lid, tweede volzin, van de Awb' -->
            artikel:  '8:75a'
                lid:  'eerste'
             volzin:  'tweede'
            lid_num:  [1]
         volzin_num:  [2]

'artikel 5:9, aanhef en onder b, Awb' -->
            artikel:  '5:9'
        aanhefonder:  'aanhef en onder b'

'artikel 8, aanhef en onder c, Wet bescherming persoonsgegevens (Wbp).' -->
            artikel:  '8'
        aanhefonder:  'aanhef en onder c'

'artikel 10, tweede lid, aanhef en onder e, van de Wob' -->
            artikel:  '10'
                lid:  'tweede'
        aanhefonder:  'aanhef en onder e'
            lid_num:  [2]

'artikel 4, tweede lid, aanhef en onder d, van het reglement van orde voor de ministerraad' -->
            artikel:  '4'
                lid:  'tweede'
        aanhefonder:  'aanhef en onder d'
            lid_num:

### Test: an actual document

In [7]:
htmldata = wetsuite.helpers.net.download('https://zoek.officielebekendmakingen.nl/stb-2001-580.html')
htmltext = ' '.join( wetsuite.helpers.split.feeling_lucky(htmldata) )  # helper function that should extract just the text

/usr/local/lib/python3.8/dist-packages/wetsuite/helpers/split.py:138: UserWarning: _split_officielepublicaties_html() needs some basic refinement
  warnings.warn("_split_officielepublicaties_html() needs some basic refinement")


In [8]:
matches = wetsuite.helpers.patterns.find_references( htmltext )
for match in matches:
    print('\nMatched: %r'%match['text'])
    for k, v in match.items():
        print( '    %20r:  %-r'%(k, v) )


Matched: 'artikel 1.1.5, onder c. 4.  Het derde lid, aanhef en onder b'
                  'type':  'artikel'
                 'start':  7064
                   'end':  7124
                  'text':  'artikel 1.1.5, onder c. 4.  Het derde lid, aanhef en onder b'
               'details':  OrderedDict([('artikel', '1.1.5'), ('lid', 'derde'), ('aanhefonder', 'aanhef en onder b'), ('lid_num', [3])])

Matched: 'artikel\r\n767'
                  'type':  'artikel'
                 'start':  8974
                   'end':  8986
                  'text':  'artikel\r\n767'
               'details':  OrderedDict([('artikel', '767')])

Matched: 'artikel 1.1.10'
                  'type':  'artikel'
                 'start':  10056
                   'end':  10070
                  'text':  'artikel 1.1.10'
               'details':  OrderedDict([('artikel', '1.1.10')])

Matched: 'artikel 1.2.3.'
                  'type':  'artikel'
                 'start':  11577
                   'end':  1159

## Marking this up in spacy

In visualizations it might be useful to mark them in the document,
as if they are (named) entities (note to self: spanruler style might make more sense).

In [10]:
import spacy, spacy.displacy
dutch = spacy.load('nl_core_news_lg')

In [12]:
test_texts = [
    # different types
    " BWBR000011  CVDR101405  CVDR101405_1 CVDR101405/1  Stb. 2011, 35;  33684R2020  ECLI:NL:HR:2005:AT4537   OJ L 69, 13.3.2013, p. 1  Council Directive 93/42/EEC of 14 June 1993   LJNs: BO6106  AO7817, NJ 2005, 270 ",
    # varied 'artikel' style cases
    '\n'.join(looser_testcases),
    # 
    'art. 2.12, eerste lid, aanhef en onder a, sub 1º artikel 79, aanhef en onder 6º artikel 3.3, zevende lid jo. vijfde lid, aanhef en onder i, Woo',
    htmltext[35500:37000], # a fragment of it, we get the idea
]

for test_text in test_texts:
    print( '-'*80 )

    doc = dutch( test_text )
    # note that you have to give exactly the same string to find_references and mark_references_spacy,
    # or the offsets will be wrong;   we use doc.text which should ensure that
    matches = wetsuite.helpers.patterns.find_references( doc.text, ljn=True) 
    wetsuite.helpers.patterns.mark_references_spacy( doc, matches )
    spacy.displacy.render( doc, style='ent', jupyter=True,
        # these are not the regular entity names displacy already chose some colors for, so:
        options={"colors":{'ECLI':'#ffaaaa', 'BWB':'#ff5577', 'CVDR':'#7755ff', 'CELEX':'#ffaaff', 'VINDPLAATS':'#aaaaff', 'EUOJ':'#aaffaa', 'EUDIR':'#aaffdd', 'LJN':'#5588ff', 'ARTIKEL':'#ffaa77'}}
    )

    #for reference in wetsuite.helpers.patterns.find_references( test_text, ljn=True ):
    #    print()
    #    pprint.pprint( reference )

--------------------------------------------------------------------------------


--------------------------------------------------------------------------------


--------------------------------------------------------------------------------


--------------------------------------------------------------------------------


## Going further?

It would be quite interesting that if 

- we resolved what it's pointing to, e.g. made input like `"artikel 4, tweede lid, aanhef en onder d, van het reglement van orde voor de ministerraad"` give any or all of:
   - BWBR0006501
   - https://wetten.overheid.nl/jci1.3:c:BWBR0006501&artikel=4    or
     https://wetten.overheid.nl/jci1.3:c:BWBR0006501&paragraaf=2&artikel=4
   - jci1.3:c:BWBR0006501&paragraaf=2&artikel=4
   - ['4', 
      '2',
      'Te dien einde beraadslaagt en besluit de raad onder meer over:', 
      'het bekendheid geven aan beleidsvoornemens in welke vorm dan ook, die van invloed kunnen zijn op de positie van het kabinet, '
      'of die belangrijke financiële consequenties kunnen hebben, benevens over beleidsvoornemens van een minister die het beleid van '
      'andere ministers kunnen raken en waarover het bereiken van overeenstemming niet mogelijk is gebleken;'
     ]

- we could deal with **document-internal references**, e.g. BWBR0004302's artikel 3, lid 1 has "artikel 2, eerste, tweede, en vijfde lid", which in that context ought to give something like:
   - ['2', 
      ['1', '2', '5'], 
      ['Bij of krachtens algemene maatregel van bestuur worden in het belang van de volksgezondheid eisen gesteld aan tabaksproducten, ',
       'elektronische dampwaar, nicotinehoudende vloeistof en niet-nicotinehoudende vloeistof met betrekking tot maximumemissieniveaus ',
       'en ingrediënten en worden technische eisen gesteld, en kunnen methoden van onderzoek worden aangewezen die bij uitsluiting ',
       'beslissend zijn voor de vaststelling of met betrekking tot een product al dan niet aan de daaraan gestelde eisen is voldaan.,

              Bij of krachtens algemene maatregel van bestuur worden in het belang van de volksgezondheid eisen gesteld aan de verpakkingseenheid en de buitenverpakking van tabaksproducten en aanverwante producten. De eisen hebben betrekking op:

          a. de aanduidingen op verpakkingseenheden en buitenverpakkingen;

          b. de hoeveelheid of het aantal stuks dat is verpakt;

          c. de presentatie van het product;

          d. de sluiting, vorm, afmetingen en materiaal van de verpakkingseenheid of de buitenverpakking; en

          e. andere elementen van de verpakkingseenheid en de buitenverpakking die gebruikt kunnen worden om een onderscheid te maken tussen de verschillende merken van een tabaksproduct of een aanverwant product.

       'De verpakkingseenheid van elektronische dampwaar bevat een bijsluiter. Onze Minister stelt nadere regels over de inhoud van de bijsluiter.'
      ]
     ]